In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import math
%matplotlib inline
plt.style.use('seaborn')

In [2]:
import sklearn
import matplotlib
import sys
import scipy
libraries = (('Matplotlib', matplotlib), ('Numpy', np), ('Pandas', pd), ('Scipy', scipy), ('Sklearn', sklearn))

print("Python Version:", sys.version, '\n')
for lib in libraries:
    print('{0} Version: {1}'.format(lib[0], lib[1].__version__))

Python Version: 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)] 

Matplotlib Version: 3.3.4
Numpy Version: 1.20.1
Pandas Version: 1.2.4
Scipy Version: 1.6.2
Sklearn Version: 0.24.1


In [3]:
def get_data():
    """
    Get some data, either pretend user data or the Iris dataset
    ------
    """
    # An oversimplified dataset for easy visualization while learning
#     headers = ['referral','country','faq','age','service']
#     my_data=[['slashdot','USA','yes',18,'None'],
#         ['google','France','yes',17,'Premium'],
#         ['google','France','yes',20,'Premium'],
#         ['reddit','USA','yes',24,'Basic'],
#         ['wiki','France','yes',23,'Basic'],
#         ['google','UK','no',21,'Premium'],
#         ['(direct)','New Zealand','no',12,'None'],
#         ['(direct)','UK','no',21,'Basic'],
#         ['google','USA','no',24,'Premium'],
#         ['slashdot','France','yes',19,'None'],
#         ['slashdot','UK','yes',31,'Basic'],
#         ['reddit','USA','no',18,'None'],
#         ['google','UK','no',18,'None'],
#         ['wiki','UK','no',19,'None'],
#         ['reddit','New Zealand','yes',12,'Basic'],
#         ['slashdot','UK','no',21,'None'],
#         ['google','UK','yes',18,'Basic'],
#         ['wiki','France','yes',19,'Basic']]
#     my_data = np.array(my_data)
#     X = my_data.T[:4]
#     y = my_data.T[-1]
#     return X.T,y.T
    # The Iris data set from SKLearn 
    # (http://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html)
    from sklearn.datasets import load_iris
    iris = load_iris()
    return iris.data, iris.target

In [4]:
import math
import numpy as np

class decision_tree_classifier:
    
    def __init__(self, max_depth = None):
        """
        Builds a decision tree to classify target data. The
        decision tree is built by trying to minimize the requested
        criteria at each possible split. Starting with the whole data
        the tree will try every possible split (column and value pair)
        and choose the split which results in the greatest reduction 
        of the criteria. Then for each sub-group of that split, the 
        process is repeated recursively until no more splits are possible
        or no splits cause a reduction of the criteria. In this class
        entropy is used as the criteria.
        ---
        KWargs:
        max_depth: how many splits to allow in the tree (depth not breadth)
        """
        self.tree = self.tree_split()
        self.max_depth = max_depth
    
    # Sub class for handling recursive nodes (only makes sense in the scope of a tree)
    class tree_split:
        """
        A sub class for handling recursive nodes. Each node will contain the value and column
        for the current split, as well as links to the resulting nodes from the split. The 
        results attribute remains empty unless the current node is a leaf. 
        """
        def __init__(self,col=-1,value=None,results=None,label=None,tb=None,fb=None):
            self.col=col # column index of criteria being tested
            self.value=value # vlaue necessary to get a true result
            self.results=results # dict of results for a branch, None for everything except endpoints
            self.tb=tb # true decision nodes 
            self.fb=fb # false decision nodes
    
    def split_data(self, X, y, colnum, value):
        """
        Returns: Two sets of data from the initial data. Set 1 contains those that passed
        the condition of data[colnum] >= value
        ----------
        Input: The dataset, the column to split on, the value on which to split
        """
        splitter = None
        if isinstance(value, int) or isinstance(value,float):
            splitter = lambda x: x[colnum] >= value
        else:
            splitter = lambda x: x[colnum] == value
        split1 = [i for i,row in enumerate(X) if splitter(row)]
        split2 = [i for i,row in enumerate(X) if not splitter(row)]
        set1X = X[split1]
        set1Y = y[split1]
        set2X = X[split2]
        set2Y = y[split2]
        return set1X, set1Y, set2X, set2Y

    def count_target_values(self, data):
        """
        Returns: A dictionary of target variable counts in the data
        """
        results = {}
        counts = np.unique(data, return_counts=True)
        for i,j in zip(*counts):
            results[i] = j
        return results

    def entropy(self, y):
        """
        Returns: Entropy of the data set, based on target values. 
        ent = Sum(-p_i Log(p_i), i in unique targets) where p is the percentage of the
        data with the ith label.
        Sidenote: We're using entropy as our measure of good splits. It corresponds to 
        information gained by making this split. If the split results in only one target type
        then the entropy new sets entropy is 0. If it results in a ton of different targets, the
        entropy will be high. 
        """
        results = self.count_target_values(y)
        log_base = len(results.keys())
        if log_base < 2:
            log_base = 2
        logb=lambda x:math.log(x)/math.log(log_base)
        ent=0.
        for r in results.keys():
            p=float(results[r])/len(y) 
            ent-=p*logb(p)
        return ent  
    
    def pandas_to_numpy(self, x):
        """
        Checks if the input is a Dataframe or series, converts to numpy matrix for
        calculation purposes.
        ---
        Input: X (array, dataframe, or series)
        Output: X (array)
        """
        if type(x) == type(pd.DataFrame()) or type(x) == type(pd.Series()):
            return x.as_matrix()
        if type(x) == type(np.array([1,2])):
            return x
        return np.array(x) 
    
    def handle_1d_data(self,x):
        """
        Converts 1 dimensional data into a series of rows with 1 columns
        instead of 1 row with many columns.
        """
        if x.ndim == 1:
            x = x.reshape(-1,1)
        return x
    
    def convert_to_array(self, x):
        """
        Takes in an input and converts it to a numpy array
        and then checks if it needs to be reshaped for us
        to use it properly
        """
        x = self.pandas_to_numpy(x)
        x = self.handle_1d_data(x)
        return x
    
    def fit(self, X, y):
        """
        Helper function to wrap the fit method. This makes sure the full nested, 
        recursively built tree gets assigned to the correct variable name and 
        persists after training.
        """
        self.tree = self._fit(X,y)
    
    def _fit(self, X, y, depth=0):
        """
        Builds the decision tree via a greedy approach, checking every possible
        branch for the best current decision. Decision strength is measured by
        information gain/entropy reduction. If no information gain is possible,
        sets a leaf node. Recursive calls to this method allow the nesting. If
        max_depth is met, all further nodes become leaves as well.
        ---
        Input: X (feature matrix), y (labels)
        Output: A nested tree built upon the node class."""
        X = self.convert_to_array(X)
        y = self.convert_to_array(y)

        if len(X) == 0: return tree_split()
        current_score = self.entropy(y)

        best_gain = 0.0
        best_criteria = None
        best_sets = None
        
        self.data_cols = X.shape[1]
        
        
        # Here we go through column by column and try every possible split, measuring the
        # information gain. We keep track of the best split then use that to send the split
        # data sets into the next phase of splitting.
        
        for col in range(self.data_cols):
            
            # find different values in this column
            column_values = set(X.T[col])
            # for each possible value, try to divide on that value
            for value in column_values:
                set1, set1_y, set2, set2_y = self.split_data(X, y, col, value)

                # Information gain
                p = float(len(set1)) / len(y)
                gain = current_score - p*self.entropy(set1_y) - (1-p)*self.entropy(set2_y)
                if gain > best_gain and len(set1_y) and len(set2_y):
                    best_gain = gain
                    best_criteria = (col, value)
                    best_sets = (np.array(set1), np.array(set1_y), np.array(set2), np.array(set2_y))
        
        
        # Now decide whether it's an endpoint or we need to split again.
        if (self.max_depth and depth < self.max_depth) or not self.max_depth:
            if best_gain > 0:
                true_branch = self._fit(best_sets[0], best_sets[1], depth=depth+1)
                false_branch = self._fit(best_sets[2], best_sets[3], depth=depth+1)
                return self.tree_split(col=best_criteria[0], value=best_criteria[1],
                        tb=true_branch, fb=false_branch)
            else:
                return self.tree_split(results=self.count_target_values(y))
        else:
            return self.tree_split(results=self.count_target_values(y))

    def print_tree(self, indent="---"):
        """
        Helper function to make sure the correct tree gets printed.
        ---
        In: indent (how to show splits between nodes)
        """
        self.__original_indent = indent
        self._print_tree_(self.tree, indent)
    
    def _print_tree_(self, tree, indent):
        """
        Goes through node by node and reports the column and value used to split
        at that node. All sub-nodes are drawn in sequence below the node.
        """
        if tree.results: # if this is a end node
            print(str(tree.results))
        else:
            print('Column ' + str(tree.col)+' : '+str(tree.value)+'? ')
            # Print the branches
            print(indent+' True: ', end=' ')
            next_indent = indent+self.__original_indent
            self._print_tree_(tree.tb,indent=next_indent)
            print(indent+' False: ', end=' ')
            self._print_tree_(tree.fb,indent=next_indent)

    def predict(self, newdata):
        """
        Helper function to make sure the correct tree is used to
        make predictions. Also manages multiple rows of input data
        since the tree must predict one at a time.
        ---
        In: new data point of the same structure as the training X.
        Out: numpy array of the resulting predictions
        """
        results = []
        for x in newdata:
            results.append(self._predict(x,self.tree))
        return np.array(results)
            
    def _predict(self, newdata, tree):
        """
        Uses the reusive structure of the tree to follow each split for
        a new data point. If the node is an endpoint, the available classes
        are sorted by "most common" and then the top choice is returned.
        """
        newdata = self.pandas_to_numpy(newdata)
        if tree.results: # if this is a end node
            return sorted(list(tree.results.items()), key=lambda x: x[1],reverse=True)[0][0]

        if isinstance(newdata[tree.col], int) or isinstance(newdata[tree.col],float):
            if newdata[tree.col] >= tree.value:
                return self._predict(newdata, tree.tb)

            else:
                return self._predict(newdata, tree.fb)
        else:
            if newdata[tree.col] == tree.value:
                return self._predict(newdata, tree.tb)
            else:
                return self._predict(newdata, tree.fb) 

    def score(self, X, y):
        """
        Uses the predict method to measure the accuracy of the model.
        ---
        In: X (list or array), feature matrix; y (list or array) labels
        Out: accuracy (float)
        """
        pred = self.predict(X)
        correct = 0
        for i,j in zip(y,pred):
            if i == j:
                correct+=1
        return float(correct)/float(len(y))

In [5]:
import pandas as pd
X,y = get_data()
print(X[10])

[5.4 3.7 1.5 0.2]


In [6]:
dt = decision_tree_classifier(max_depth=5)
dt.fit(pd.DataFrame(X),pd.Series(y))

AttributeError: 'DataFrame' object has no attribute 'as_matrix'